## Uplift Random Forest

https://habr.com/ru/companies/ru_mts/articles/485976/
https://www.youtube.com/watch?v=u69k2d-7E_0

In [6]:
np.log2(4)

2.0

In [7]:
import numpy as np
def kl_divergence(pk, qk):
    '''
    Calculate KL Divergence for binary classification.

    sum(np.array(pk) * np.log(np.array(pk) / np.array(qk)))

    Args
    ----
    pk : float
        The probability of 1 in one distribution.
    qk : float
        The probability of 1 in the other distribution.

    Returns
    -------
    S : float
        The KL divergence.
    '''

    eps: cython.float = 1e-6
    S: cython.float

    if qk == 0.:
        return 0.

    qk = min(max(qk, eps), 1 - eps)

    if pk == 0.:
        S = -np.log2(1 - qk)
    elif pk == 1.:
        S = -np.log2(qk)
    else:
        S = pk * np.log2(pk / qk) + (1 - pk) * np.log2((1 - pk) / (1 - qk))

    return S

In [8]:
kl_divergence(0.5, 0.5)

0.0

In [14]:
np.log2(0.1)

-3.321928094887362

In [13]:
kl_divergence(1.0, 0.1)

3.321928094887362

For debug

In [16]:
from pathlib import Path
import sys
 
if sys.argv:
    sys.path.insert(0, str(Path('/Users/ogrobertino/UpliftModelingResearch/').resolve()))
%load_ext autoreload
%autoreload 2

In [73]:
parameters = {
    'n_estimators': (50,130),
    'max_depth': (20, 50),
    'min_samples_leaf': (50, 150),
    'n_reg': (1, 100),
    'evaluationFunction': "KL"
}

In [61]:
# from src.utils import get_paths_train_test, train_test_model
# from src.factory import UpliftRandomForestModelFactory
# from src.models.CausalML.Models import UpliftRandomForestModel
# from tqdm import tqdm
# configs = UpliftRandomForestModel.generate_config(count=1, **parameters)
# ds_name = 'lazada'
# features_percent = 50
# factory = UpliftRandomForestModelFactory
# config = configs[0]
# batch_size=32
# max_size=100000
# train_path, test_path = get_paths_train_test(ds_name=ds_name, features_percent=features_percent)
# model, train, test = factory.create(config, train_path, test_path)

1. Сделать ноутбук с отбором фичей: Хочу обрезать количеством фичей у lazadы (можно взять много моделек, пересечь их импортансы)
2. Мб добавить Qiny метрику
3. Для всех моделек в трейне надо сделать kfold / забутстрапить и хранить помимо метрики еще и среднее.
4. Добавить пруиннг для случайного леса после того как получится завести случайный лес)
5. Попробовать бустинг на аплифт-деревьях https://github.com/jd-opensource/UTBoost

In [ ]:
from src.utils import get_paths_train_test, train_test_model
from src.factory import UpliftRandomForestModelFactory
from src.models.CausalML.Models import UpliftRandomForestModel
from tqdm import tqdm

configs = UpliftRandomForestModel.generate_config(count=10, **parameters)

for i in tqdm(range(len(configs)), desc="Training models"):
    train_test_model(ds_name='lazada', features_percent=100, factory=UpliftRandomForestModelFactory, config=configs[i])    

Training models:   0%|                                    | 0/10 [00:00<?, ?it/s]

In [75]:
from src.global_params import BASE_PATH
import pandas as pd
table = pd.read_csv(BASE_PATH / "exps/stats.tsv", sep='\t')
table[table['Model'] == 'UpliftRandomForestModel']

,Model,Path,Dataset,Features Percent,Latency (ms),Binary Size (MB),AUUC (test),Precision@5,Precision@10,Precision@15,Precision@20,Precision@25,Precision@50,Compressions
105,UpliftRandomForestModel,exps/lazada/100/108,lazada,100,0.497881,3.176101,0.475090,-0.008102,0.001116,0.001797,0.001116,0.001315,0.003999,{}
106,UpliftRandomForestModel,exps/lazada/100/109,lazada,100,0.523351,0.553565,0.420009,-0.005352,-0.000650,0.000807,-0.000650,0.002197,0.003062,{}
107,UpliftRandomForestModel,exps/lazada/100/110,lazada,100,0.852198,9.715878,0.497662,-0.004999,-0.001060,0.000797,-0.001060,0.001365,0.004678,{}
108,UpliftRandomForestModel,exps/lazada/100/111,lazada,100,0.932907,19.550749,0.504164,-0.007807,0.000699,0.001920,0.000699,0.002933,0.004328,{}
109,UpliftRandomForestModel,exps/lazada/100/112,lazada,100,0.570844,8.116217,0.485961,-0.004271,-0.002005,-0.000502,-0.002005,0.001619,0.004571,{}
110,UpliftRandomForestModel,exps/lazada/100/113,lazada,100,0.928169,18.951253,0.432151,-0.005172,-0.001156,-0.000115,-0.001156,0.001737,0.002764,{}
111,UpliftRandomForestModel,exps/lazada/100/114,lazada,100,1.707120,43.066350,0.442632,-0.004530,-0.000203,0.001026,-0.000203,0.001717,0.003313,{}


In [13]:
# table.sort_values(by=['Precision@10'], ascending=False)

In [9]:
table.tail(5)

,Model,Path,Dataset,Features Percent,Latency (ms),Binary Size (MB),AUUC (test),Precision@5,Precision@10,Precision@15,Precision@20,Precision@25,Precision@50,Compressions
92,TModel,exps/lazada/100/95,lazada,100,0.059551,15.731801,0.621811,0.008838,0.007589,0.006579,0.007589,0.005857,0.004556,{}
93,SModel,exps/lazada/100/96,lazada,100,0.029183,0.121993,0.586844,0.008476,0.005866,0.005930,0.005866,0.005800,0.005237,{}
94,TModel,exps/lazada/100/97,lazada,100,0.060034,54.043945,0.461269,-0.002724,0.005485,0.005803,0.005485,0.006601,0.003238,{}
95,XModel,exps/lazada/100/98,lazada,100,0.036554,23.910056,0.413690,0.004096,0.004087,0.002589,0.004087,0.002975,0.003292,{}
96,XModel,exps/lazada/100/99,lazada,100,0.037296,1.417968,0.487874,0.003950,0.006380,0.006161,0.006380,0.005778,0.003796,{}


In [ ]:
from src.models.ICausalML.Models import XModel
path = pd.read_csv(BASE_PATH / "exps/stats.tsv", sep='\t').Path.iloc[-1]
model = XModel(from_load=True, path = str(BASE_PATH) + "/" + path)